In [1]:
#!pip install -q transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 113.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 80.9 MB/s eta 0:00:00


In [27]:
import pandas as pd
from transformers import pipeline

In [28]:
twitter_df = pd.read_csv("TwitterDataSentiment.csv")
news_df = pd.read_csv("NewsDataSentiment.csv")

In [29]:
twitter_df.tail()

,date,clean_text,ticker
78144,10/13/2022,now you can trade dxy on bingxenglish bingxeng...,DXY
78145,10/13/2022,nebraskangooner do i get an apology now im jus...,DXY
78146,10/12/2022,ecb more hawkish from here while the fed is co...,DXY
78147,10/11/2022,dxy monster intraday reclaim,DXY
78148,10/11/2022,dxy starting a daily eq,DXY


In [30]:
news_df.tail()

,ticker,company,description,date
66980,OGS,"ONE Gas, Inc.","Alton Williams, Mark Long, Ruthie Alcaide and ...",03/16/2021
66981,OGS,"ONE Gas, Inc.",The sports network mixed up the Georgia native...,03/02/2021
66982,OGS,"ONE Gas, Inc.",Onyx returns with thier first explosive single...,02/17/2021
66983,OGS,"ONE Gas, Inc.","Ramona Singer, one of the OGs from The Real Ho...",02/03/2021
66984,OGS,"ONE Gas, Inc.",He claims he wasn't speaking about veterans wh...,01/06/2021


In [31]:
# pretrained bert models

twitter_model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
news_model    = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone")

Device set to use cuda:0
Device set to use cuda:0


In [32]:
# Set up pipeline using GPU (device=0)
twitter_model = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment", device=0)

# Get texts
texts = twitter_df['clean_text'].tolist()

# Run inference in batches
twitter_preds = twitter_model(texts, batch_size=64)

# Assign predictions
twitter_df['sentiment_label'] = [pred['label'] for pred in twitter_preds]
twitter_df['sentiment_score'] = [pred['score'] for pred in twitter_preds]

Device set to use cuda:0


In [35]:
label_map = {
    'LABEL_0': 'Negative',
    'LABEL_1': 'Neutral',
    'LABEL_2': 'Positive'
}

twitter_df['sentiment'] = twitter_df['sentiment_label'].map(label_map)

In [40]:
# Set up FinBERT pipeline using GPU (device=0)
news_model = pipeline("sentiment-analysis", model="yiyanghkust/finbert-tone", device=0)

# Get the text column
news_texts = news_df['description'].dropna().tolist()  # drop nulls to avoid errors

# Run inference in batches
news_preds = news_model(news_texts, batch_size=64)

# Assign predictions
news_df['sentiment_label'] = [pred['label'] for pred in news_preds]
news_df['sentiment_score'] = [pred['score'] for pred in news_preds]

Device set to use cuda:0


In [44]:
news_df.head(15)

,ticker,company,description,date,sentiment_label,sentiment_score
0,AAPL,Apple Inc.,These are the stocks posting the largest moves...,04/11/2025,Neutral,0.999930
1,AAPL,Apple Inc.,Tech giant Apple (AAPL) is planning to send mo...,04/08/2025,Neutral,0.999819
2,AAPL,Apple Inc.,Apple’s (AAPL) market capitalization has falle...,04/08/2025,Negative,0.998158
3,AAPL,Apple Inc.,These are the stocks posting the largest moves...,04/07/2025,Neutral,0.999930
4,AAPL,Apple Inc.,A total of $11.1 trillion has been wiped off t...,04/05/2025,Neutral,0.999938
5,AAPL,Apple Inc.,These are the stocks posting the largest moves...,01/31/2025,Neutral,0.998953
6,AAPL,Apple Inc.,Analysts on Wall Street are responding favorab...,01/31/2025,Positive,1.000000
7,AAPL,Apple Inc.,"The report was filed on January 31, 2025",01/31/2025,Neutral,0.999983
8,AAPL,Apple Inc.,Analysts are concerned about how Apple will si...,01/30/2025,Negative,1.000000
9,AAPL,Apple Inc.,These are the stocks posting the largest moves...,01/21/2025,Neutral,0.999930
